In [1]:
from assignee import *
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
from tqdm import tqdm
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import os
import requests
import json
import time
from dotenv import load_dotenv
load_dotenv()
engine = establish_connection()

ASSIGNEE_TYPE_DICT = {
    1: "Unassigned",
    2: "United States company or corporation",
    3: "Foreign company or corporation",
    4: "United States individual",
    5: "Foreign individual",
    6: "U.S. Federal government",
    7: "Foreign government",
    8: "U.S. county government",
    9: "U.S. state government"
}

## 01 - Sample

In [35]:
size=800
output_dir="data/"
seed = 20231025

In [4]:
# Load in data
disamb = pd.read_csv(
    "g_persistent_assignee.tsv.zip",
    dtype=str,
    sep="\t",
    compression="zip")

In [21]:
# Clean data
disamb["mention_id"] = "US" + disamb["patent_id"] + "-" + disamb["assignee_sequence"]
disamb_20230629 = disamb[["mention_id", "disamb_assignee_id_20230629"]]
disamb_20230629 = disamb_20230629.dropna()

In [38]:
# Sample and extract cluster sizes
sampled_df = disamb_20230629.sample(n=size, random_state=seed)
cluster_size_lookup = disamb_20230629["disamb_assignee_id_20230629"].value_counts()
sampled_df["cluster_size"] = [cluster_size_lookup[disamb_id] for disamb_id in sampled_df["disamb_assignee_id_20230629"]]
sampled_df

,mention_id,disamb_assignee_id_20230629,cluster_size
1553328,USD964454-0,5706771b-83bd-4f2d-862e-bf0ea0232c54,4
5510832,US9552680-0,63f0315e-177e-45c6-b1e5-08ea61d1f86b,23036
5460323,US11091722-0,1204ec27-e8d6-47d4-92fc-ed124d96403c,757
3908499,US9820494-0,852ccd2f-d989-4d82-ac8e-7bbfaa6ac41f,63
5181285,US5031150-0,20f6d656-e62e-4430-94a3-ec3a1a7d5e4b,51991
...,...,...,...
7438133,US9432930-0,9079525d-53a3-4322-9023-47e276cc7f0a,42903
5587875,US9467607-0,3729a868-e503-44c3-89f3-17dc1e6b9964,48
690367,USD696749-0,7f85306d-2bfb-498d-b473-4b48bdc507a6,132
4250826,US4895107-0,20f6d656-e62e-4430-94a3-ec3a1a7d5e4b,51991


In [40]:
# Save output
np.savetxt(os.path.join(output_dir, "01 - sample.txt"), sampled_df["mention_id"].values, fmt="%s")
sampled_df[["mention_id", "cluster_size"]].to_csv(os.path.join(output_dir, "01 - sample_with_cluster_size.csv"), index=False)

## 02 - Populate Sample

In [42]:
with engine.connect() as connection:
    populate_sample(connection=connection, sample_path="data/01 - sample.txt", output_path="data/02 - sample_with_data.csv")

OperationalError: (pymysql.err.OperationalError) (1356, "View 'algorithms_assignee_labeling.assignee' references invalid table(s) or column(s) or function(s) or definer/invoker of view lack rights to use them")
[SQL: SELECT patent_id, assignee_sequence, organization, name_first, name_last, assignee_type, title, patent_date,            assignee_country, assignee_city, assignee_state FROM algorithms_assignee_labeling.assignee WHERE            patent_id='D964454' and assignee_sequence='0']
(Background on this error at: https://sqlalche.me/e/20/e3q8)

## 03 - Segement Sample
Take the populated sample from Step 02 and split into `n` different files where `n` is the number of hand labelers. Let's assume `n=3` for now.

In [ ]:
segment_sample()

# 04 - Match Organizations
This step is done using streamlit.

# 05 - Extraction

In [19]:
assignee_disambiguation_IDs = ["6e8e8a6c-a5ef-485c-adf7-cb0e654e8e8b", "3c02a0b9-06d5-4486-850a-93680e052484"]

In [20]:
id_list = '("' + '","'.join(assignee_disambiguation_IDs) + '")'
query = f"SELECT * FROM algorithms_assignee_labeling.assignee a WHERE a.disambiguated_assignee_id IN {id_list}"
with engine.connect() as connection:
    result = connection.execute(text(query)).fetchall()
df = pd.DataFrame(result).drop_duplicates()

In [ ]:
"""
Parameters
----------
ws : openpyxl worksheet
    Worksheet for appending rows from 'df' and merging like cells
index_start : int
    First index of the mention_id group
index_end : int
    Last index of the mention_id group
num_columns : int
    Number of columns to apply the merge cell function

Returns
-------
ws : openpyxl worksheet
    Updated worksheet with all rows from `df` to `ws`
"""
def merge_cells(ws, index_start, index_end, num_columns):
    # Loop over columns
    for i in range(1, num_columns + 1):

        # Loop over rows in pre-defined range
        start = index_start
        end = start
        for j in range(index_start, index_end + 1):

            if j == start: # Initialize the previous value at the start of the section
                prev_value = ws.cell(column=i, row=j).value
            elif ws.cell(column=i, row=j).value == prev_value:
                if j == index_end: # Consecutive cells with same value need to merge if end of section
                    ws.merge_cells(start_row=start, end_row=j, start_column=i, end_column=i)
                else: # Otherwise, consecutive cells with same value need to check the next cell before merging
                    end += 1
            elif start != end: # Merge prior sequence once a different value is in place
                ws.merge_cells(start_row=start, end_row=end, start_column=i, end_column=i)
                start = j + 1
                end = j + 1

In [21]:
# Create a new Excel workbook and add a worksheet
wb = Workbook()
ws = wb.active

# Loop through mention_id groups and sort values
for group, data in tqdm(df.groupby(['patent_id', 'assignee_sequence']), desc="Assignee Mention Loop"):
    df_temp = data.sort_values(by=["inventor_id", "cpc_subgroup_id"], inplace=False)

    # Determine indexing values and add rows to worksheets
    index_start = ws.max_row + 1
    index_end = index_start + len(df_temp.index) - 1
    for row in dataframe_to_rows(df_temp, index=False, header=(index_start == 2)):
        ws.append(row)

    merge_cells(ws, index_start, index_end, len(df.columns))

Assignee Mention Loop: 100%|██████████| 2/2 [00:00<00:00, 27.66it/s]


# X - Testing for streamlit

In [5]:
query = "SELECT * FROM algorithms_assignee_labeling.assignee WHERE assignee.organization='Lutron Electronics Co., Inc.'"
with engine.connect() as connection:
    results = connection.execute(text(query)).fetchall()
pd.DataFrame(results)

[('D477289', 'Switch', None, datetime.date(2003, 7, 15), 'design', 'S1', '63f4ddc7-1e20-4ea4-ac21-48b20d6c634d', 'Lutron Electronics Co., Inc.', None, None, 0, 2, 'Coopersburg', 'PA', 'US', '2001/29151527', '29/151527', 'US', '29151527', '29151527', '29', datetime.date(2001, 11, 13), datetime.date(2003, 7, 15), None, None, None, None, None, None, None, None, None, 'fl:no_ln:mayo-2', 0, 'Noel', 'Mayo', 'Philadelphia', 'PA', 'US'),
 ('D477289', 'Switch', None, datetime.date(2003, 7, 15), 'design', 'S1', '63f4ddc7-1e20-4ea4-ac21-48b20d6c634d', 'Lutron Electronics Co., Inc.', None, None, 0, 2, 'Coopersburg', 'PA', 'US', '2001/29151527', '29/151527', 'US', '29151527', '29151527', '29', datetime.date(2001, 11, 13), datetime.date(2003, 7, 15), None, None, None, None, None, None, None, None, None, 'fl:ja_ln:adams-10', 1, 'Jason O.', 'Adams', 'Emmaus', 'PA', 'US'),
 ('D477289', 'Switch', None, datetime.date(2003, 7, 15), 'design', 'S1', '63f4ddc7-1e20-4ea4-ac21-48b20d6c634d', 'Lutron Electronic

# Testing Merged Cells

In [2]:
assignee_disambiguation_IDs = ["a0ba1f5c-6e5f-4f62-b309-22bd81c8b043"] # ["a0ba1f5c-6e5f-4f62-b309-22bd81c8b043", "e1d5391e-94c9-4ced-843b-6992e29b6fee", "8f703249-da60-44ea-a257-fb6a07b08f50"]
id_list = '("' + '","'.join(assignee_disambiguation_IDs) + '")'
query = f"SELECT * FROM algorithms_assignee_labeling.assignee a WHERE a.disambiguated_assignee_id IN {id_list}"

In [36]:
keep_cols = ['mention_id', 'title', 'abstract', 'patent_date', 'patent_type', 'patent_kind',
       'disambiguated_assignee_id', 'organization', 'name_first', 'name_last', 'assignee_type',
       'assignee_city', 'assignee_state', 'assignee_country', 'inventor_sequence', 'inventor_id',
       'inventor_name_first', 'inventor_name_last', 'inventor_citie', 'inventor_state',
       'inventor_country', 'cpc_section_id', 'cpc_group_id', 'cpc_group_title']

merge_groupings = [
       {"primary": "mention_id",
       "merge_group": [
              'mention_id', 'title', 'abstract', 'patent_date', 'patent_type', 'patent_kind',
              'disambiguated_assignee_id', 'organization', 'name_first', 'name_last', 'assignee_type',
              'assignee_city', 'assignee_state', 'assignee_country'
       ]},
       {"primary": "inventor_sequence",
        "merge_group": [
              'inventor_sequence', 'inventor_id', 'inventor_name_first', 'inventor_name_last',
              'inventor_citie', 'inventor_state', 'inventor_country'
       ]},
       {"primary": "cpc_section_id", "merge_group": ["cpc_section_id", "cpc_group_id", "cpc_group_title"]},
]

In [37]:
with engine.connect() as connection:
    result = connection.execute(text(query)).fetchall()
df = pd.DataFrame(result)
df["mention_id"] = df.patent_id.str.cat(df.assignee_sequence.astype(str), sep='-')
df['assignee_type'] = df['assignee_type'].map(ASSIGNEE_TYPE_DICT)
df = df[keep_cols]
df.drop_duplicates(inplace=True)
df.sort_values(['mention_id', 'inventor_sequence', 'cpc_group_id']).to_csv('test_base.csv')

## In-house method

In [38]:
def merge_group(ws, start, end, grouping):
    for column in grouping:
        col_index = keep_cols.index(column) + 1
        ws.merge_cells(start_row=start, end_row=end, start_column=col_index, end_column=col_index)

def merge_cells(ws, index_start, index_end, num_columns):
    # Loop over columns
    for grouping in merge_groupings:
        col_index = keep_cols.index(grouping['primary']) + 1

        # Merge range variables
        merge_start = index_start
        merge_end = index_start
        prev_value = ws.cell(column=col_index, row=index_start).value

        # Loop over rows in pre-defined range
        for j in range(index_start + 1, index_end + 1):

            if ws.cell(column=col_index, row=j).value == prev_value:
                if j == index_end: # Consecutive cells with same value need to merge if end of section
                    merge_group(ws=ws, start=merge_start, end=j, grouping=grouping['merge_group'])
                    # ws.merge_cells(start_row=merge_start, end_row=j, start_column=i, end_column=i)
                else: # Otherwise, consecutive cells with same value need to only expand merge range
                    merge_end = j
            elif merge_start != merge_end: # Merge prior sequence once a different value is in place
                merge_group(ws=ws, start=merge_start, end=merge_end, grouping=grouping['merge_group'])
                # ws.merge_cells(start_row=merge_start, end_row=merge_end, start_column=i, end_column=i)
                merge_start = j
                merge_end = j
                prev_value = ws.cell(column=col_index, row=j).value

In [39]:
# Create a new Excel workbook and add a worksheet
wb = Workbook()
ws = wb.active

# Loop through mention_id groups and sort values
for group, data in tqdm(df.groupby('mention_id'), "Outer groupby loop"):

    df_temp = data.sort_values(["inventor_sequence", "cpc_group_id"], ascending=True)

    # Determine indexing values and add rows to worksheets
    index_start = ws.max_row + 1
    index_end = index_start + len(df_temp.index) - 1
    for row in dataframe_to_rows(df_temp, index=False, header=(index_start == 2)):
        ws.append(row)

    merge_cells(ws, index_start, index_end, len(df.columns))
    
wb.save('test.xlsx')

Outer groupby loop:   0%|          | 0/526 [00:00<?, ?it/s]

Outer groupby loop: 100%|██████████| 526/526 [01:31<00:00,  5.76it/s]


## Groupby

## Set Index

# PV API for Extraction Step

In [3]:
load_dotenv()
api_key = os.getenv('pv_api_key')

Here we'll load in a dictionary of our CPC group titles so that we can map the codes to the titles.

In [ ]:
# Get our CPC titles
base_url = 'https://search.patentsview.org'
endpoint = 'api/v1/cpc_subclass'
data = {'size': 1000}
responses = []

while True:
    response = requests.get(f"{base_url}/{endpoint}/?o={json.dumps(data)}", headers={"X-Api-Key": api_key})
    if response.status_code == 429:
        wait_for = response.headers['Retry-After']
        time.sleep(wait_for)
        continue
    if response.status_code != 200:
        raise Exception(response.headers)
    response_data = response.json()
    responses += response_data['cpc_subclasses']
    expected = response_data['total_hits']
    if (response_data['count'] == 0) or (len(responses) >= expected):
        break
    data['after'] = response_data['cpc_subclasses'][-1]['cpc_subclass_id']

cpc_subclass = {item['cpc_subclass_id']: item['cpc_subclass_title'] for item in responses}

API Search Tool: https://search.patentsview.org/swagger-ui/

PV API example: https://github.com/PatentsView/PatentsView-Code-Snippets/blob/master/07_Elasticsearch_API_demo/PV%20ES%20API%20tutorial.ipynb

After-before example: https://github.com/PatentsView/PatentsView-Search-API/blob/7561b789b523f230563fcfba16dcee3f4c19d090/API/tests/SearchAPITest.py#L13

In the following code, we'll define the parameters of our initial API call to the patents endpoint for our first query. Since the output size is limited, we'll have to update the 'o' paramter for consecutive runs (similar to how we extract our cpc_subclass dictionary above). For now, let's just get some sample output for the first 10 results.

In [117]:
# Input
assignee_disambiguation_IDs = ["a0ba1f5c-6e5f-4f62-b309-22bd81c8b043"]

# Query Options
base_url = 'https://search.patentsview.org'
endpoint = 'api/v1/patent'
param_dict = {
    "f" : ["patent_id", "patent_title", "patent_abstract", "patent_date", "patent_type", "assignees.*",\
           "inventors.*", "cpc_current.*"],
    "o" : {"size":10},
    "q" : {"assignees.assignee_id":assignee_disambiguation_IDs},
    "s" : [{"patent_id":"asc"}],
}

# Getting final query
param_string = "&".join([f"{param_name}={json.dumps(param_val)}" for param_name, param_val in param_dict.items()])
query_url = f"{base_url}/{endpoint.strip('/')}/?{param_string}"
response = requests.get(query_url, headers={"X-Api-Key": api_key})
output = response.json()['patents']

{'error': False,
 'count': 10,
 'total_hits': 526,
 'patents': [{'patent_id': '10141424',
   'patent_title': 'Method of producing a channel structure formed from a plurality of strained semiconductor bars',
   'patent_type': 'utility',
   'patent_date': '2018-11-27',
   'patent_abstract': 'Method of manufacturing a structure with semiconducting bars suitable for forming one at least one transistor channel, including the following steps:',
   'assignees': [{'assignee': 'https://search.patentsview.org/api/v1/assignee/7e2456c4-4663-469d-9d9c-0d73de45cccd/',
     'assignee_type': '7',
     'assignee_individual_name_first': None,
     'assignee_individual_name_last': None,
     'assignee_organization': "COMMISSARIAT A L'ÉNERGIE ATOMIQUE ET AUX ÉNERGIES ALTERNATIVES",
     'assignee_city': 'Paris',
     'assignee_state': None,
     'assignee_country': 'FR',
     'assignee_sequence': 0},
    {'assignee': 'https://search.patentsview.org/api/v1/assignee/a0ba1f5c-6e5f-4f62-b309-22bd81c8b043/',
 

Now we'll define a method to get all of our output from the PV API.

In [138]:
# Requires that base_url, endpoint, and api_key are all defined
def full_extraction_output(assignee_disambiguation_IDs):
    full_output = []
    param_dict = {
        "f" : ["patent_id", "patent_title", "patent_abstract", "patent_date", "patent_type", "assignees.*",\
            "inventors.*", "cpc_current.*"],
        "o" : {"size":1000},
        "q" : {"assignees.assignee_id":assignee_disambiguation_IDs},
        "s" : [{"patent_id":"asc"}],
    }
    
    while True:
        # Update query parameters and get response
        param_string = "&".join([f"{param_name}={json.dumps(param_val)}" for param_name, param_val in param_dict.items()])
        query_url = f"{base_url}/{endpoint.strip('/')}/?{param_string}"
        response = requests.get(query_url, headers={"X-Api-Key": api_key})

        # Check for errors
        if response.status_code == 429:
            wait_for = response.headers['Retry-After']
            time.sleep(wait_for)
            continue
        if response.status_code != 200:
            raise Exception(response.headers)

        response_data = response.json()
        full_output += response_data['patents']
        expected = response_data['total_hits']
        if (response_data['count'] == 0) or (len(full_output) >= expected):
            break
        param_dict["o"]['after'] = response_data['patents'][-1]['patent_id']
        
    return full_output

Here we'll define helper methods for processing our API request and creating our df.

In [102]:
def new_assignees(row):
    if 'assignees' in row.keys():
        new_assignees = []
        for assignee in row['assignees']:
            assignee['assignee'] = assignee['assignee'][47:-1]
            if assignee['assignee'] in assignee_disambiguation_IDs:
                assignee['assignee_type'] = ASSIGNEE_TYPE_DICT[int(assignee['assignee_type'])]
                new_assignees.append(assignee)
        return new_assignees
    else: # Empty assignees
        return [{'assignee': '', 'assignee_type': '', 'assignee_individual_name_first': '', 'assignee_individual_name_last': '',\
            'assignee_organization': '', 'assignee_city': '', 'assignee_state': '', 'assignee_country': '', 'assignee_sequence': ''}]

def new_inventors(row):
    # Extract disamb_ID from URL for inventors
    if 'inventors' in row.keys():
        inventors = row['inventors'].copy()
        for inventor in inventors:
            inventor['inventor'] = inventor['inventor'][47:-1]
        return inventors
    else: # Empty inventors
        return [{'inventor': '', 'inventor_name_first': '', 'inventor_name_last': '', 'inventor_city': '',\
            'inventor_state': '', 'inventor_country': '', 'inventor_sequence': ''}]

def new_cpc(row):
    # Handle cases with empty cpc_current   
    if 'cpc_current' in row.keys():
        new_cpc = []
        subclasses_observed = set()
        for cpc in row['cpc_current']:
            subclass_id = cpc['cpc_subclass_id']
            if subclass_id not in subclasses_observed:
                subclasses_observed.add(subclass_id)
                new_cpc.append({'cpc_section': subclass_id[0], 'cpc_subclass_id': subclass_id, 'cpc_subclass_title': cpc_subclass[subclass_id]})
        return new_cpc
    else:
        return [{'cpc_section': '', 'cpc_subclass_id': '', 'cpc_subclass_title': ''}]
    
def process_json(json_data):
    for row in json_data:
        row['assignees'] = new_assignees(row)
        row['inventors'] = new_inventors(row)
        row['cpc_current'] = new_cpc(row)

Now let's organize our results into one CSV file. First we'll define some helper methods.

In [135]:
def add_empty_rows(df, max_rows):
    num_empty_rows = max_rows - len(df)
    empty_rows = pd.DataFrame([{}]*num_empty_rows, columns=df.columns)
    return pd.concat([df, empty_rows], ignore_index=True)

def parition_endpoints(result):
    # Create separate dataframes
    inventors = pd.DataFrame.from_dict(result['inventors']).sort_values('inventor_sequence')
    cpc = pd.DataFrame.from_dict(result['cpc_current']).sort_values('cpc_subclass_id')
    assignees = pd.DataFrame.from_dict(result['assignees']).sort_values('assignee_sequence')

    # Add empty rows
    row_count = max(len(inventors.index), len(cpc.index), len(assignees.index))
    inventors = add_empty_rows(inventors, row_count)
    cpc = add_empty_rows(cpc, row_count)
    assignees = add_empty_rows(assignees, row_count)
    
    return inventors, cpc, assignees

def extraction_output_to_csv(output, output_path):
    df_list = []
    patent_fields = ["patent_id", "patent_title", "patent_abstract", "patent_date", "patent_type"]

    for result in output:
        # Separating data to add empty rows where necessary
        inventors, cpc, assignees = parition_endpoints(result)
        patent = {field: result[field] for field in patent_fields}
        patent_df = pd.DataFrame.from_dict([patent] * max(len(inventors.index), len(cpc.index), len(assignees.index)))

        # Combining and storing result
        result = pd.concat([patent_df, assignees, inventors, cpc], axis=1)
        df_list.append(result)
        
    pd.concat(df_list).to_csv(output_path)

Instead of appending each result df to the df_list, we can use the following merge cell code with an activate worksheet.

In [ ]:
wb = Workbook()
ws = wb.active

merge_start = ws.max_row + 1
merge_end = merge_start + len(result.index) - 1
for row in dataframe_to_rows(result, index=False, header=(merge_start == 2)):
    ws.append(row)
for col in range(1, 6):
    ws.merge_cells(start_row=merge_start, end_row=merge_end, start_column=col, end_column=col)

wb.save('output.xlsx')

In [137]:
output = full_extraction_output(assignee_disambiguation_IDs=assignee_disambiguation_IDs)

526


In [139]:
process_json(output)

In [140]:
extraction_output_to_csv(output, "output.csv")

# Extraction for Siji

In [6]:
assignee_IDs = np.loadtxt('data/US5031150-0.txt', dtype="str").tolist()

In [9]:
PATENT_FIELDS = ["patent_id", "patent_title", "patent_abstract", "patent_date", "patent_type"]
f_list = PATENT_FIELDS + ["assignees.*"]
API_KEY = os.getenv('pv_api_key')
BASE_URL = 'https://search.patentsview.org'

In [10]:
full_output = []
endpoint = 'api/v1/patent'
param_dict = {
    "f" : f_list,
    "o" : {"size":1000},
    "q" : {"assignees.assignee_id": assignee_IDs},
    "s" : [{"patent_id":"asc"}],
}

In [22]:
param_string = "&".join([f"{param_name}={json.dumps(param_val)}" for param_name, param_val in param_dict.items()])
query_url = f"{BASE_URL}/{endpoint.strip('/')}/?{param_string}"
response = requests.get(query_url, headers={"X-Api-Key": API_KEY})

In [28]:
post_url = f"{BASE_URL}/{endpoint.strip('/')}"
headers = {"X-Api-Key": API_KEY, "Content-Type": "application/json"}
response = requests.post(post_url, data=json.dumps(param_dict), headers=headers)

In [32]:
response.json()

{'error': False,
 'count': 1000,
 'total_hits': 62883,
 'patents': [{'patent_id': '10000070',
   'patent_title': 'Printer and printer control method',
   'patent_type': 'utility',
   'patent_date': '2018-06-19',
   'patent_abstract': 'According to one embodiment, a thermal head is provided opposite to a platen roller. A ribbon transport unit transports an ink ribbon between the thermal head and the platen roller. A transport unit transports a sheet carrying a label between the thermal head and the platen roller. A clamping mechanism clamps the ink ribbon, a printing area of the label and the sheet in the thermal head and the platen roller. An acquisition unit acquires first print data to be printed on a first label and second print data to be printed on a second label. A non-printing area specifying unit specifies a non-printing area on the sheet, based on the first print data and the second print data. The control unit separates the ink ribbon and the sheet when it is transported betw

# Compare

In [10]:
review = pd.read_csv('data/06 - compare/US5031150-0-difference.csv', dtype={'patent_id': str})
print("Table size:", len(review.index))
review

Table size: 69450


,patent_id,patent_title,patent_abstract,patent_date,patent_type,assignee,assignee_type,assignee_individual_name_first,assignee_individual_name_last,assignee_organization,assignee_city,assignee_state,assignee_country,assignee_sequence,Siddharth,Siji
0,10000070,Printer and printer control method,"According to one embodiment, a thermal head is...",2018-06-19,utility,4bc11268-4b2d-4b57-a966-2faeb1701ae8,Foreign company or corporation,NaN,NaN,Toshiba Tec Kabushiki Kaisha,Tokyo,NaN,JP,0,True,False
1,10001285,Outdoor unit of air conditioner,"According to one embodiment, an object of the ...",2018-06-19,utility,8d71907f-4488-4d49-8793-39a46f9f0a42,Foreign company or corporation,NaN,NaN,TOSHIBA CARRIER CORPORATION,Yokohama,NaN,JP,0,True,False
2,10001458,"Eddy current flaw detection device, eddy curre...",There is provided an eddy current flaw detecti...,2018-06-19,utility,20f6d656-e62e-4430-94a3-ec3a1a7d5e4b,Foreign company or corporation,NaN,NaN,KABUSHIKI KAISHA TOSHIBA,Minato,NaN,JP,0,True,False
3,10001524,Semiconductor integrated circuit and test meth...,"According to one embodiment, a semiconductor i...",2018-06-19,utility,20f6d656-e62e-4430-94a3-ec3a1a7d5e4b,Foreign company or corporation,NaN,NaN,KABUSHIKI KAISHA TOSHIBA,Tokyo,NaN,JP,0,True,True
4,10001534,Magnetic resonance imaging apparatus and radio...,"According to one embodiment, an MRI apparatus ...",2018-06-19,utility,286eba76-8b49-416d-bed5-17b3e7396631,Foreign company or corporation,NaN,NaN,TOSHIBA MEDICAL SYSTEMS CORPORATION,Tochigi,NaN,JP,0,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69445,RE48664,Electronic device,"According to one embodiment, an electronic dev...",2021-07-27,reissue,67e2f4c8-acfe-4a65-b37b-7891db690cb3,Foreign company or corporation,NaN,NaN,Toshiba Memory Corporation,Minato,NaN,JP,0,True,True
69446,RE48939,Semiconductor memory card,"According to one embodiment, a semiconductor m...",2022-02-22,reissue,20f6d656-e62e-4430-94a3-ec3a1a7d5e4b,Foreign company or corporation,NaN,NaN,KABUSHIKI KAISHA TOSHIBA,Minato,NaN,JP,0,True,True
69447,RE48952,Recording medium controller and method thereof,"According to one embodiment, a recording mediu...",2022-03-01,reissue,20f6d656-e62e-4430-94a3-ec3a1a7d5e4b,Foreign company or corporation,NaN,NaN,KABUSHIKI KAISHA TOSHIBA,Tokyo,NaN,JP,0,True,True
69448,RE48983,Memory device and controlling method of the same,A memory device includes a memory which has me...,2022-03-22,reissue,67e2f4c8-acfe-4a65-b37b-7891db690cb3,Foreign company or corporation,NaN,NaN,Toshiba Memory Corporation,Minato,NaN,JP,0,True,True


# 05 - Extraction (resolved)

In [1]:
from sqlalchemy import create_engine, text
from dotenv import dotenv_values
import pandas as pd

ASSIGNEE_TYPE_DICT = {
    1: "Unassigned",
    2: "United States company or corporation",
    3: "Foreign company or corporation",
    4: "United States individual",
    5: "Foreign individual",
    6: "U.S. Federal government",
    7: "Foreign government",
    8: "U.S. county government",
    9: "U.S. state government"
}

In [2]:
def get_engine():
    config = dotenv_values(".env")
    user = config['user']
    pswd = config['password']
    hostname = config['hostname']
    dbname = config['dbname']
    engine = create_engine(f"mysql+pymysql://{user}:{pswd}@{hostname}/{dbname}?charset=utf8mb4")
    return engine

engine = get_engine()

In [3]:
with engine.connect() as connection:
    query = "SELECT * FROM rawassignee_for_hand_labeling r where assignee = '9e98dbf5-7cc3-42fe-a6ad-8cd62eda7372';"
    result = connection.execute(text(query)).fetchall()

In [4]:
df = pd.DataFrame(result)
df['assignee_type'] = df['assignee_type'].apply(lambda x: ASSIGNEE_TYPE_DICT[x])
df[['patent_id', 'assignee_sequence', 'patent_title', 'patent_abstract', 'patent_date', 'patent_type',\
    'assignee', 'assignee_type', 'assignee_individual_name_first', 'asassignee_individual_name_last',\
    'assignee_organization', 'assignee_city', 'assignee_state', 'assignee_country']]

,patent_id,assignee_sequence,patent_title,patent_abstract,patent_date,patent_type,assignee,assignee_type,assignee_individual_name_first,asassignee_individual_name_last,assignee_organization,assignee_city,assignee_state,assignee_country
0,6018738,0,Methods and apparatus for matching entities an...,"Matching (e.g., via correlation or similarity...",2000-01-25,utility,9e98dbf5-7cc3-42fe-a6ad-8cd62eda7372,United States company or corporation,None,None,Microsft Corporation,Redmond,WA,US
1,6025837,0,Electronic program guide with hyperlinks to ta...,An interactive entertainment system has a pro...,2000-02-15,utility,9e98dbf5-7cc3-42fe-a6ad-8cd62eda7372,United States company or corporation,None,None,Micrsoft Corporation,Redmond,WA,US
2,5794197,0,Senone tree representation and evaluation,A speech recognition method provides improved...,1998-08-11,utility,9e98dbf5-7cc3-42fe-a6ad-8cd62eda7372,United States company or corporation,None,None,Micrsoft Corporation,Redmond,WA,US
3,6909883,0,Wireless communication device,A receiving section has one crystal resonator ...,2005-06-21,utility,9e98dbf5-7cc3-42fe-a6ad-8cd62eda7372,Foreign company or corporation,None,None,"Micro FT Co., Ltd.",Tokyo,None,JP
4,6754386,0,Method and system of matching ink processor an...,In a computing device that receives handwritte...,2004-06-22,utility,9e98dbf5-7cc3-42fe-a6ad-8cd62eda7372,United States company or corporation,None,None,Microsft Corporation,Redmond,WA,US
5,6686883,0,Antenna,A transmission antenna is formed on a transmis...,2004-02-03,utility,9e98dbf5-7cc3-42fe-a6ad-8cd62eda7372,Foreign company or corporation,None,None,"Micro FT Co., Ltd.",Tokyo,None,JP
6,D574848,0,Icon for a portion of a display screen,None,2008-08-12,design,9e98dbf5-7cc3-42fe-a6ad-8cd62eda7372,United States company or corporation,None,None,Microosft Corporation,Redmond,WA,US
7,7587668,0,Using existing content to generate active cont...,A computer implemented method of converting ex...,2009-09-08,utility,9e98dbf5-7cc3-42fe-a6ad-8cd62eda7372,United States company or corporation,None,None,Microft Corporation,Redmond,WA,US
8,8667010,0,Database table partitioning allowing overlaps ...,Various embodiments provide a set of algorithm...,2014-03-04,utility,9e98dbf5-7cc3-42fe-a6ad-8cd62eda7372,United States company or corporation,None,None,Microsfot Corporation,Redmond,WA,US
9,7058224,0,Detecting multiple objects in digital image data,Apparatus and methods for detecting multiple i...,2006-06-06,utility,9e98dbf5-7cc3-42fe-a6ad-8cd62eda7372,United States company or corporation,None,None,Microsfot Corporation,Redmond,WA,US
